# <h1 style="text-align: center; font-family: Arial, sans-serif;">Проект по исследованию поведения пользователей мобильного приложения "Ненужные вещи"</h1>


## Шаг 1. Общая информация и переработка

In [1]:
# загрузка требуемых для работы библиотек
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
import re
import os
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
# загрузка файлов
try:
    mobile_dataset = pd.read_csv(r'G:\aleksei_I\datasets_for_git_hub\mobile_dataset.csv')
    mobile_sources = pd.read_csv(r'G:\aleksei_I\datasets_for_git_hub\mobile_sourсes.csv')
except:
    mobile_dataset = pd.read_csv('https://code.s3.yandex.net/datasets/mobile_dataset.csv')
    mobile_sources = pd.read_csv('https://code.s3.yandex.net/datasets/mobile_sources.csv')

### Подробнее про `mobile_dataset` и его переработка

In [3]:
# функция для вывода общей информации и поиска дубликатов в датасете
def data_info(data):
    # Вывод основной информации о данных
    display("Информация о данных:")
    display(data.info())


    # Вывод первых строк
    display("Первые строки данных:")
    display(data.head())


    # Поиск и вывод дубликатов
    duplicated_rows = data[data.duplicated()]
    if not duplicated_rows.empty:
        display("Найдены дубликаты:")
        display(duplicated_rows)
    else:
        display("Дубликатов не найдено.")

In [4]:
data_info(mobile_dataset)

'Информация о данных:'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74197 entries, 0 to 74196
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   event.time  74197 non-null  object
 1   event.name  74197 non-null  object
 2   user.id     74197 non-null  object
dtypes: object(3)
memory usage: 1.7+ MB


None

'Первые строки данных:'

,event.time,event.name,user.id
0,2019-10-07 00:00:00.431357,advert_open,020292ab-89bc-4156-9acf-68bc2783f894
1,2019-10-07 00:00:01.236320,tips_show,020292ab-89bc-4156-9acf-68bc2783f894
2,2019-10-07 00:00:02.245341,tips_show,cf7eda61-9349-469f-ac27-e5b6f5ec475c
3,2019-10-07 00:00:07.039334,tips_show,020292ab-89bc-4156-9acf-68bc2783f894
4,2019-10-07 00:00:56.319813,advert_open,cf7eda61-9349-469f-ac27-e5b6f5ec475c


'Дубликатов не найдено.'

In [5]:
# camel_case для названий колонок
mobile_dataset.rename(
    columns={
        'event.time': 'event_time', 
        'event.name': 'event_name', 
        'user.id': 'user_id'
    }, 
    inplace=True
)


In [6]:
# приведение столбца event_time к типу `datetime` и округление до секунд
mobile_dataset['event_time'] = pd.to_datetime(mobile_dataset['event_time']).dt.round('1s')

In [7]:
# проверка дат
display(mobile_dataset['event_time'].min())
display(mobile_dataset['event_time'].max())

Timestamp('2019-10-07 00:00:00')

Timestamp('2019-11-03 23:58:13')

In [8]:
# проверка на соответствие с описанием данных столбца `event_name`
mobile_dataset['event_name'].unique()

array(['advert_open', 'tips_show', 'map', 'contacts_show', 'search_4',
       'search_5', 'tips_click', 'photos_show', 'search_1', 'search_2',
       'search_3', 'favorites_add', 'contacts_call', 'search_6',
       'search_7', 'show_contacts'], dtype=object)

In [9]:
# объединим одинаковые по своей сути действия 'show_contacts' и `contacts_show`
mobile_dataset['event_name'] = mobile_dataset['event_name'].replace({'contacts_show': 'show_contacts'}, inplace=False)

Поскольку в рамках данного исследования разнообразие действий связанных с поиском `search_n` нам не понадобятся, переименуем все разнообразие показателей в `search`

In [10]:
mobile_dataset.loc[mobile_dataset['event_name'].str.contains('search'), 'event_name'] = 'search'

In [11]:
# проверка изменений
display("Названия колонок в переменной mobile_dataset:")
display(mobile_dataset.columns)

display("Тип данных в столбце 'event_time':")
display(mobile_dataset['event_time'].dtype)

display("Уникальные значения в столбце 'event_name':")
display(mobile_dataset['event_name'].unique())

'Названия колонок в переменной mobile_dataset:'

Index(['event_time', 'event_name', 'user_id'], dtype='object')

"Тип данных в столбце 'event_time':"

dtype('<M8[ns]')

"Уникальные значения в столбце 'event_name':"

array(['advert_open', 'tips_show', 'map', 'show_contacts', 'search',
       'tips_click', 'photos_show', 'favorites_add', 'contacts_call'],
      dtype=object)

### Подробнее про `mobile_sources.csv` и его переработка

In [12]:
data_info(mobile_sources)

'Информация о данных:'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4293 entries, 0 to 4292
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   userId  4293 non-null   object
 1   source  4293 non-null   object
dtypes: object(2)
memory usage: 67.2+ KB


None

'Первые строки данных:'

,userId,source
0,020292ab-89bc-4156-9acf-68bc2783f894,other
1,cf7eda61-9349-469f-ac27-e5b6f5ec475c,yandex
2,8c356c42-3ba9-4cb6-80b8-3f868d0192c3,yandex
3,d9b06b47-0f36-419b-bbb0-3533e582a6cb,other
4,f32e1e2a-3027-4693-b793-b7b3ff274439,google


'Дубликатов не найдено.'

In [13]:
# camel_case для названий колонок
mobile_sources.rename(columns={'userId': 'user_id'}, inplace=True)

In [14]:
# проверка изменений
mobile_sources.columns

Index(['user_id', 'source'], dtype='object')



---



In [15]:
# объединим датафреймы в один и проверим уникальные значения в результирующем фрейме
data = mobile_dataset.merge(mobile_sources, left_on= 'user_id', right_on= 'user_id',how='left')

data.apply(lambda x: len(x.unique())).sort_values()

source            3
event_name        9
user_id        4293
event_time    71378
dtype: int64

In [16]:
# проверка дубликатов
data[data.duplicated(subset=['event_time', 'event_name', 'user_id'], keep=False)].sort_values(by='user_id')

,event_time,event_name,user_id,source
39297,2019-10-22 20:49:43,photos_show,00753c79-ea81-4456-acd0-a47a23ca2fb9,yandex
39296,2019-10-22 20:49:43,photos_show,00753c79-ea81-4456-acd0-a47a23ca2fb9,yandex
37906,2019-10-22 13:02:27,map,007d031d-5018-4e02-b7ee-72a30609173f,yandex
37905,2019-10-22 13:02:27,map,007d031d-5018-4e02-b7ee-72a30609173f,yandex
8111,2019-10-10 14:46:37,tips_show,017c6afc-965d-4c94-84ee-f0e326998e30,other
...,...,...,...,...
57728,2019-10-28 22:25:49,search,ff82c7c7-16d6-44b1-833b-a217747b0b02,yandex
1932,2019-10-07 20:32:50,photos_show,ffc01466-fdb1-4460-ae94-e800f52eb136,yandex
1931,2019-10-07 20:32:50,photos_show,ffc01466-fdb1-4460-ae94-e800f52eb136,yandex
54109,2019-10-27 20:00:18,tips_show,fffb9e79-b927-4dbb-9b48-7fd09b23a62b,google


In [17]:
# сброс дубликатов
data.drop_duplicates(subset=['event_time', 'event_name', 'user_id'], keep='first', inplace=True)

In [18]:
# ищем пропуски
data.isnull().sum()

event_time    0
event_name    0
user_id       0
source        0
dtype: int64

## Промежуточный **итог** по первому шагу.

> Были проведенедены следующие манипуляции с данными:
 - колонки переименованы для приведения их к общему виду,
 - тип данных в столбце с датой приведен к соответствующему типу, данные округлены до секунд,
 - данные о действиях связанных с поиском обьеденены в одно действие `search`,
 - действия связанные с показом контактов объеденены в одно,
 - датафреймы объеденены.

> данные за период с 03-11-2019 по 07-10-2019

## Шаг 2. Исследовательский анализ

### Выделение сессий

В исслеловании Adjust среднее время сессии пользователя в сегменте "Маркетплейсы и доски объявлений" определено в 13 минут.

[Ссылка на статью](https://traff.ink/articles/trendy-mobilnyh-prilozhenij-2021-2022/#sessians)



In [19]:
# Сортировка действий по времени и сброс индексов
data = data.sort_values(by=['user_id', 'event_time']).reset_index(drop=True)

# Таймаут в 5 минут
g = (data.groupby('user_id')['event_time'].diff() > pd.Timedelta('5Min')).cumsum()

# Создание столбца с идентификатором сессий
data['session_id'] = data.groupby(['user_id', g], sort=False).ngroup() + 1

In [20]:
# Проверка длительности сессии при таймауте в 5 минут

# Создание столбца с продолжительностью сессии
data['session_duration'] = data.groupby(['user_id', 'session_id'])['event_time'].transform(lambda x: x.max() - x.min())

display(f"Средняя продолжительность сессий:", data['session_duration'].mean())

'Средняя продолжительность сессий:'

Timedelta('0 days 00:12:10.641157697')

Таймаут в 5 минут выбран верно, поскольку средняя родолжительность составила 12 минут, что близко к значению в [исследовании Adjust](https://traff.ink/articles/trendy-mobilnyh-prilozhenij-2021-2022/#sessians)

### Поиск сценариев

In [21]:
# Группируем данные по session_id и объединяем последовательность действий в одну строку
scenarios = data.groupby('session_id')['event_name'].apply(lambda x: ', '.join(x)).reset_index()

# Переименовываем столбец
scenarios.columns = ['session_id', 'scenario']

# Функция для удаления всех повторяющихся действий из сценариев
def remove_duplicates(scenario):
    unique_actions = []
    for action in scenario.split(', '):
        if action not in unique_actions:
            unique_actions.append(action)
    return ', '.join(unique_actions)

# Применяем функцию к столбцу 'scenario'
scenarios['scenario'] = scenarios['scenario'].apply(remove_duplicates)



In [22]:
# Фильтрация сценариев, заканчивающихся показом контактов с действиями больше двух
filtered_scenarios = scenarios[scenarios['scenario'].str.endswith('show_contacts') & (scenarios['scenario'].str.count(',') > 1)]

# Подсчет количества каждого сценария
top_scenarios = filtered_scenarios['scenario'].value_counts().head(3)
top_scenarios

scenario
map, tips_show, show_contacts         82
search, photos_show, show_contacts    28
search, tips_show, show_contacts      22
Name: count, dtype: int64

### Воронки сценариев

Определили три оновных сценария по которым построим воронки в разрезе уникальных пользователей. При построении воронки будет выяснено у какого сценария лучше конверсия в целевое действие.

In [23]:
# Выбираем первый сценарий из топа
scenario = top_scenarios.index[0]

# Разбиваем сценарий на отдельные действия
actions = scenario.split(', ')

# Создаем список для хранения данных воронки
funnel_data = []

# Подсчитываем количество уникальных пользователей на каждом этапе воронки
for i, action in enumerate(actions):
    users_at_step = set(data[data['event_name'] == action]['user_id'])
    if i == 0:
        funnel_data.append({'step': action, 'count': len(users_at_step)})
    else:
        prev_users = set(data[data['event_name'] == actions[i - 1]]['user_id'])
        users_at_step = users_at_step.intersection(prev_users)
        funnel_data.append({'step': action, 'count': len(users_at_step)})

# Построение воронки с помощью Plotly
fig = go.Figure()

fig.add_trace(go.Funnel(
    name='Сценарий',
    y=[step['step'] for step in funnel_data],
    x=[step['count'] for step in funnel_data],
    textinfo="value+percent previous",
    marker=dict(color="skyblue"),
    textposition="inside",
    textfont=dict(family="Arial", size=12, color="white")))

fig.update_layout(
    title=f"Воронка сценария '{scenario}' в разрезе уникальных пользователей",
    font=dict(family="Arial", size=12),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    height=600
)

fig.show()


In [24]:
# Выбираем первый сценарий из топа
scenario = top_scenarios.index[1]

# Разбиваем сценарий на отдельные действия
actions = scenario.split(', ')

# Создаем список для хранения данных воронки
funnel_data = []

# Подсчитываем количество уникальных пользователей на каждом этапе воронки
for i, action in enumerate(actions):
    users_at_step = set(data[data['event_name'] == action]['user_id'])
    if i == 0:
        funnel_data.append({'step': action, 'count': len(users_at_step)})
    else:
        prev_users = set(data[data['event_name'] == actions[i - 1]]['user_id'])
        users_at_step = users_at_step.intersection(prev_users)
        funnel_data.append({'step': action, 'count': len(users_at_step)})

# Построение воронки с помощью Plotly
fig = go.Figure()

fig.add_trace(go.Funnel(
    name='Сценарий',
    y=[step['step'] for step in funnel_data],
    x=[step['count'] for step in funnel_data],
    textinfo='value+percent previous',
    marker=dict(color="skyblue"),
    textposition="inside",
    textfont=dict(family="Arial", size=12, color="white")))

fig.update_layout(
    title=f"Воронка сценария '{scenario}' в разрезе уникальных пользователей",
    font=dict(family="Arial", size=12),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    height=600
)

fig.show()


In [25]:
# Выбираем первый сценарий из топа
scenario = top_scenarios.index[2]

# Разбиваем сценарий на отдельные действия
actions = scenario.split(', ')

# Создаем список для хранения данных воронки
funnel_data = []

# Подсчитываем количество уникальных пользователей на каждом этапе воронки
for i, action in enumerate(actions):
    users_at_step = set(data[data['event_name'] == action]['user_id'])
    if i == 0:
        funnel_data.append({'step': action, 'count': len(users_at_step)})
    else:
        prev_users = set(data[data['event_name'] == actions[i - 1]]['user_id'])
        users_at_step = users_at_step.intersection(prev_users)
        funnel_data.append({'step': action, 'count': len(users_at_step)})

# Построение воронки с помощью Plotly
fig = go.Figure()

fig.add_trace(go.Funnel(
    name='Сценарий',
    y=[step['step'] for step in funnel_data],
    x=[step['count'] for step in funnel_data],
    textinfo='value+percent previous',
    marker=dict(color="skyblue"),
    textposition="inside",
    textfont=dict(family="Arial", size=12, color="white")))

fig.update_layout(
    title=f"Воронка сценария '{scenario}' в разрезе уникальных пользователей",
    font=dict(family="Arial", size=12),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    height=600
)

fig.show()


### Вывод по анализу воронок

 - Второй сценарий имеет самую высокую общую конверсию в просмотр контактов (show_contacts) после этапа показа фотографий (photos_show).
Однако, сценарий 3 также выглядит достаточно привлекательным, особенно если учесть, что конверсия на этапе показа рекомендованных объявлений (tips_show) выше, чем в первом сценарии.

 - Первый сценарий имеет низкую конверсию на этапе просмотра контактов (show_contacts), возможно из-за невысокой конверсии на предыдущем этапе (tips_show).

 - Таким образом, на основе предоставленных данных, можно сделать вывод, что сценарий 3 обладает наилучшей конверсией в целевое действие "просмотр контактов", за счет более высокой конверсии на этапе предъявления рекомендованных объявлений. Однако, второй сценарий также представляет собой хороший вариант благодаря высокой конверсии после показа фотографий.

### Оценка действий, совершеных пользователями, которые просматривают контакты

In [26]:
# Получение списка пользователей, которые просматривали контакты
users_with_contacts_show = data[data['event_name'] == 'show_contacts']['user_id'].unique()

# Фильтрация данных только для пользователей, которые просматривали контакты
filtered_data = data[data['user_id'].isin(users_with_contacts_show)]

# Подсчет количества событий для каждого типа
event_counts = filtered_data.groupby('event_name').size().reset_index(name='count')

# Подсчет общего числа событий
total_events = event_counts['count'].sum()

# Выделение топ-5 категорий
top_events = event_counts.sort_values(by='count', ascending=False).head(5)

# Создание категории "другие" для остальных событий
other_count = total_events - top_events['count'].sum()
other_row = pd.DataFrame({'event_name': ['Other'], 'count': [other_count]})
event_counts = pd.concat([top_events, other_row])

# Рассчет процентного соотношения
event_counts['percent'] = (event_counts['count'] / total_events * 100).round(2)

# Создание столбчатой диаграммы
fig = go.Figure(data=[go.Bar(x=event_counts['event_name'], y=event_counts['percent'])])

# Настройка макета
fig.update_layout(
    title="Процентное соотношение топ-5 событий и 'других' для пользователей, которые просматривали контакты",
    xaxis=dict(title='Событие'),
    yaxis=dict(title='Процент'),
    font=dict(family="Arial", size=12),
    height=600
)

fig.update_traces(texttemplate='%{y:.2f}%', textposition='outside')

# Отображение диаграммы
fig.show()

### Оценка действий, совершеных пользователями, которые не просматривают контакты

In [27]:
# Получение списка пользователей, которые НЕ просматривали контакты
users_without_contacts_show = data[~data['user_id'].isin(users_with_contacts_show)]['user_id'].unique()

# Фильтрация данных только для пользователей, которые НЕ просматривали контакты
filtered_data_without_contacts = data[data['user_id'].isin(users_without_contacts_show)]

# Подсчет количества событий для каждого типа
event_counts_without_contacts = filtered_data_without_contacts.groupby('event_name').size().reset_index(name='count')

# Подсчет общего числа событий
total_events_without_contacts = event_counts_without_contacts['count'].sum()

# Выделение топ-5 категорий
top_events_without_contacts = event_counts_without_contacts.sort_values(by='count', ascending=False).head(5)

# Создание категории "другие" для остальных событий
other_count_without_contacts = total_events_without_contacts - top_events_without_contacts['count'].sum()
other_row_without_contacts = pd.DataFrame({'event_name': ['Other'], 'count': [other_count_without_contacts]})
event_counts_without_contacts = pd.concat([top_events_without_contacts, other_row_without_contacts])

# Рассчет процентного соотношения
event_counts_without_contacts['percent'] = (event_counts_without_contacts['count'] / total_events_without_contacts * 100).round(2)

# Создание столбчатой диаграммы
fig = go.Figure(data=[go.Bar(x=event_counts_without_contacts['event_name'], y=event_counts_without_contacts['percent'])])

# Настройка макета
fig.update_layout(
    title="Процентное соотношение топ-5 событий и 'других' для пользователей, которые НЕ просматривали контакты",
    xaxis=dict(title='Событие'),
    yaxis=dict(title='Процент'),
    font=dict(family="Arial", size=12),
    height=600
)

fig.update_traces(texttemplate='%{y:.2f}%', textposition='outside')

# Отображение диаграммы
fig.show()


### Вывод по оценке действий групп пользователей, просматревших контакты и не просмотревших контакты.

 - Сравнительный анализ действий пользователей, совершивших просмотр контактов, и пользователей, не совершивших его, указывает на незначительные различия в их поведении на платформе. Обе группы пользователей демонстрируют сходные предпочтения и шаблоны взаимодействия:

 - Просмотр фото: В обеих группах просмотр фотографий является наиболее распространенным действием. Это подтверждает важность визуальной информации для пользователей при принятии решения о недвижимости.

 - Поиск и фильтрация: Обе группы пользователей проявляют активность в поиске и фильтрации объявлений с целью нахождения наиболее подходящих вариантов.

 - Открытие объявлений: Открытие обычных объявлений также остается популярным действием как среди пользователей, просмотревших контакты, так и среди тех, кто этого не сделал.

 - Эти сходства свидетельствуют о том, что ключевые этапы взаимодействия пользователей с платформой остаются неизменными независимо от того, просматривают ли они контакты или нет.

## Шаг 3.Проверка гипотез

### Гипотеза о различиях в конверсии двух групп пользователей совершивших определенные действия

 > **Н0:** Различий в конверсиях в просмотры контактов conacts_show, между группой пользователей совершившими tips_show - tips_click и только tips_show нет.

 > **Н1:** Есть различия в конверсиях в просмотры контактов conacts_show, между группой пользователей совершившими tips_show - tips_click и только tips_show.

#### Подготовка данных

In [28]:
# создаем пустые фреймы для разделения на группы
data_tips_show_click = pd.DataFrame()
data_tips_show = pd.DataFrame()

'''
группируем данные 'data' по столбцу 'user_id', если в группе есть и событие 'tips_click',
и событие 'tips_show', то данные этой группы добавляются в data_tips_show_click.
В противном случае, если в группе есть только событие 'tips_show', данные добавляются в data_tips_show.
'''
for name, group in data.groupby('user_id'):
    if 'tips_click' in group['event_name'].values and 'tips_show' in group['event_name'].values:
        data_tips_show_click = pd.concat([data_tips_show_click, group])
    elif 'tips_show' in group['event_name'].values:
        data_tips_show = pd.concat([data_tips_show, group])

data_tips_show_click.reset_index(drop=True, inplace=True)
data_tips_show.reset_index(drop=True, inplace=True)

# количество уникальных пользователей в группах data_tips_show_click и data_tips_show
group_a_nuniq = data_tips_show_click['user_id'].nunique()
group_b_nuniq = data_tips_show['user_id'].nunique()

# количество уникальных пользователей в группах, совершивших событие 'show_contacts'.
group_a_nuniq_show = data_tips_show_click[data_tips_show_click['event_name'] == 'show_contacts']['user_id'].nunique()
group_b_nuniq_click = data_tips_show[data_tips_show['event_name'] == 'show_contacts']['user_id'].nunique()

display('Уникальных пользователей совершивших `tips_show` и `tips_click`', group_a_nuniq)
display('Уникальных пользователей совершивших `tips_show`', group_b_nuniq)
display('Уникальных пользователей группы `tips_show` и `tips_click`, совершивших целевое действие', group_a_nuniq_show)
display('Уникальных пользователей группы `tips_show`, совершивших целевое действие', group_b_nuniq_click)

'Уникальных пользователей совершивших `tips_show` и `tips_click`'

297

'Уникальных пользователей совершивших `tips_show`'

2504

'Уникальных пользователей группы `tips_show` и `tips_click`, совершивших целевое действие'

91

'Уникальных пользователей группы `tips_show`, совершивших целевое действие'

425

In [29]:
# конверсии в группах
conversion_group_a = group_a_nuniq_show / group_a_nuniq
conversion_group_b = group_b_nuniq_click / group_b_nuniq

#### Тест

In [30]:
display('Конверсия в группе `tips_show` - `tips_click`', conversion_group_a)
display('Конверсия в группе `tips_show`', conversion_group_b)

# Задаем данные для теста
successes = np.array([group_a_nuniq_show, group_b_nuniq_click])
nobs = np.array([group_a_nuniq, group_b_nuniq])

# Выполняем z-тест для двух долей
stat, pval = proportions_ztest(successes, nobs)


# Выводим результаты теста
display(f'p-value: {pval}')

alpha = 0.05
if pval < alpha:
    display('Отвергаем нулевую гипотезу: есть статистически значимая разница в конверсии.')
else:
    display('Не можем отвергнуть нулевую гипотезу: нет статистически значимой разницы в конверсии.')

'Конверсия в группе `tips_show` - `tips_click`'

0.3063973063973064

'Конверсия в группе `tips_show`'

0.16972843450479233

'p-value: 9.218316568768822e-09'

'Отвергаем нулевую гипотезу: есть статистически значимая разница в конверсии.'

### Гипотеза о различиях конверсии групп пользователей пришедших из разных источников


 > **Н0:** Нет статистически значимых различий в конверсиях в просмотры контактов conacts_show, групп пользователей установивших приложение из источника google и yandex.

 > **H1:** Есть статистически значимые различия в конверсиях в просмотры контактов conacts_show, групп пользователей установивших приложение из источника google и yandex.

#### Подготовка данных

In [31]:
# Выделение выборок
google_users = data[data['source'] == 'google']
yandex_users = data[data['source'] == 'yandex']

google_unique_users = google_users.drop_duplicates(subset='user_id')
yandex_unique_users = yandex_users.drop_duplicates(subset='user_id')

# Фильтрация данных
google_show_contacts_users = google_unique_users[google_unique_users['event_name'] == 'show_contacts']
yandex_show_contacts_users = yandex_unique_users[yandex_unique_users['event_name'] == 'show_contacts']

# Подсчет количества пользователей в группах
google_total_users = google_unique_users['user_id'].nunique()
yandex_total_users = yandex_unique_users['user_id'].nunique()

# Подсчет количества пользователей, совершивших целевое действие в группах
google_show_contacts_total = google_show_contacts_users['user_id'].nunique()
yandex_show_contacts_total = yandex_show_contacts_users['user_id'].nunique()

# Вычисление конверсии
conversion_google = (google_show_contacts_total) / (google_total_users) * 100
conversion_yandex = (yandex_show_contacts_total) / (yandex_total_users) * 100

#### Тест

In [32]:
display("Конверсия в просмотры контактов для пользователей из Google: {:.2f}%".format(conversion_google))
display("Конверсия в просмотры контактов для пользователей из Yandex: {:.2f}%".format(conversion_yandex))

successes = np.array([google_show_contacts_total, yandex_show_contacts_total])
nobs = np.array([google_total_users, yandex_total_users])

z_stat, p_value = proportions_ztest(successes, nobs)

# Выводим результаты теста
display(f'p-value: {p_value}')

alpha = 0.05
if p_value < alpha:
    display('Отвергаем нулевую гипотезу: есть статистически значимая разница в конверсии.')
else:
    display('Не можем отвергнуть нулевую гипотезу: нет статистически значимой разницы в конверсии.')


'Конверсия в просмотры контактов для пользователей из Google: 4.52%'

'Конверсия в просмотры контактов для пользователей из Yandex: 4.65%'

'p-value: 0.8621384071819487'

'Не можем отвергнуть нулевую гипотезу: нет статистически значимой разницы в конверсии.'

In [33]:
successes, nobs

(array([51, 90]), array([1129, 1934]))

## Выводы по проверке гипотез

1. Проведенный тест на равенство конверсий в просмотры контактов между группой пользователей, совершивших действия tips_show и tips_click, и группой пользователей, совершивших только tips_show, показал отсутствие статистически значимых различий в конверсиях (p >= 0.05). Таким образом, нет оснований отвергать нулевую гипотезу о том, что различий в конверсиях в просмотры контактов между этими двумя группами нет.

2. Проведенный Z-тест на равенство конверсий в просмотры контактов между группами пользователей, установивших приложение из источника Google и Yandex, показал статистически значимые различия в конверсиях (p < 0.05). Таким образом, отвергается нулевая гипотеза о том, что нет статистически значимых различий в конверсиях между группами пользователей из разных источников.

## Общий вывод

1. **Управление вовлеченностью клиентов**:
   - Исследование поведения пользователей в мобильном приложении "Ненужные вещи" дало ценные данные о предпочтениях и шаблонах взаимодействия пользователей.
   - Анализ воронок сценариев позволил выявить наиболее привлекательные сценарии для пользователей, что является основой для улучшения приложения с точки зрения пользовательского опыта.

2. **Оценка групп пользователей**:
   - Сравнительный анализ действий пользователей, просмотревших контакты и не просмотревших их, показал, что основные этапы взаимодействия с платформой остаются неизменными.
   - Незначительные различия в поведении пользователей указывают на то, что улучшение конверсии в просмотр контактов может потребовать более тонкой настройки на другие аспекты, кроме предложения контактов.

3. **Проверка гипотез**:
   - Проведенные тесты на равенство конверсий подтвердили отсутствие статистически значимых различий в конверсиях между группами пользователей, совершившими определенные действия.
   - Так же статистически значимые различия в конверсиях между группами пользователей, пришедших из разных источников тест не выявил.

Рекомендации:
- **Персонализация контента**: Необходимо разработать стратегии персонализации контента, чтобы более точно соответствовать предпочтениям каждой группы пользователей.
- **Оптимизация источников привлечения**: Имея в виду различия в конверсиях между группами пользователей из разных источников, стоит оптимизировать стратегии привлечения пользователей из этих источников с учетом этих различий.
- **Дальнейшее исследование**: Непрерывное мониторинг и анализ поведения пользователей позволит более точно адаптировать приложение и улучшать его в соответствии с изменяющимися потребностями пользователей.
